In [1]:
! pip install polars

In [2]:
! pip install lightgbm

In [3]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 

dataPath = "data/"

In [4]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    for col in df.columns:
        if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
            df = df.with_columns(pl.col(col).cast(pl.Int32))
        elif col in ["date_decision"] or col[-1] in ("D"):
            df = df.with_columns(pl.col(col).cast(pl.Date))
        elif col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64))
        elif col[-1] in ("M"):
            df = df.with_columns(pl.col(col).cast(pl.String))
        else: #(L, T)
            df = df.with_columns(pl.col(col).cast(pl.Float32))
    
    return df

In [5]:
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv").pipe(set_table_dtypes)
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_a_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_a_1_1.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_a_1_2.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_a_1_3.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)

In [6]:
# [case_id 그룹별 aggregation 함수]

# case_id별로 aggregation 후, 해당 값으로 group 내의 결측치를 채움
# group 전체에 결측치만 존재한다면, 컬럼 전체의 median으로 결측치를 채움
def groupby_agg_method(df: pl.DataFrame, col_name: str, agg_method:str):
    subset = df.select(['case_id', col_name])
    
    if str == 'mean':
        count = subset.group_by('case_id').agg(pl.mean(col_name).alias(col_name))
    elif str == 'max':
        count = subset.group_by('case_id').agg(pl.max(col_name).alias(col_name))
    else:
        # 우선적으로 에러가 나지 않게 하기 위함
        count = subset.group_by('case_id').agg(pl.mean(col_name).alias(col_name))
    
    merged_df = subset.join(count, on='case_id', how='left')
    print(merged_df)
    
    new_col_name = f'{col_name}_right'
    merged_df = merged_df.with_columns(pl.col(new_col_name), pl.col(new_col_name).is_null().map_elements(lambda x: 1 if x else 0).alias(f"{col_name}_is_null"))
    
    df = df.hstack(merged_df.select(f"{col_name}_is_null"))
    df = df.with_columns(pl.when(merged_df[new_col_name].is_null()).then(df[col_name].median()).otherwise(merged_df[new_col_name]).alias(col_name))
    
    
    return df, df[col_name]

In [7]:
# [case_id 그룹별 aggregation 함수 - T]
# 그룹별 mode 값 선정 
# 만약 mode가 여러개인 경우(최빈값 lst 길이 계산), 무작위 선택(nominal value)
def groupby_mode_value(df:pl.DataFrame, col_name:str):
    def list_length(lst):
        return len(lst)

    mode_value = train_static.select(['case_id', col_name]) \
        .drop_nulls() \
        .group_by('case_id') \
        .agg(pl.col(col_name).mode().alias('mode_value')) \
        .with_columns(pl.col('mode_value').map_elements(list_length).alias('mode_value_length'))

    
    mode_get_value = train_static.select(['case_id', col_name]) \
        .drop_nulls() \
        .group_by('case_id') \
        .agg((pl.col(col_name).mode()).sample(n=1, seed=42).get(0).alias('mode_value'))

    return mode_get_value

In [8]:
def groupby_first_value(df:pl.DataFrame, col_name:str):
    subset = df.select(['case_id', col_name])
    
    count = subset.group_by('case_id').agg(pl.first(col_name).alias(col_name))
    merged_df = subset.join(count, on='case_id', how='left')
    print(merged_df)
    
    new_col_name = f'{col_name}_right'
    df = df.with_columns(pl.when(merged_df[new_col_name].is_null()).then(0).otherwise(merged_df[new_col_name]).alias(col_name))

    return df

In [9]:
groupping_mean_lst = ['annualeffectiverate_199L', 'annualeffectiverate_63L',
                      'contractsum_5085717L','interestrate_508L', 'nominalrate_281L', 'nominalrate_498L', 'numberofcontrsvalue_258L', 'numberofcontrsvalue_358L', 'numberofinstls_320L', 'numberofinstls_229L','numberofoutstandinstls_59L', 
                      'numberofoutstandinstls_520L', 'numberofoverdueinstlmax_1039L', 'numberofoverdueinstls_834L', 'prolongationcount_1120L', 'prolongationcount_599L',
                      'credlmt_230A', 'credlmt_935A', 'debtoutstand_525A', 'debtoverdue_47A','instlamount_768A', 'instlamount_852A', 'monthlyinstlamount_332A', 'monthlyinstlamount_674A', 
                      'outstandingamount_362A', 'outstandingamount_354A', 'overdueamount_659A', 'overdueamount_31A', 'residualamount_488A','residualamount_856A',  
                     'dpdmax_139P', 'dpdmax_757P']
groupping_max_lst = ['numberofoverdueinstlmax_1151L', 'numberofoverdueinstls_725L', 'overdueamountmax2_14A', 'overdueamountmax2_398A']

groupping_total_lst = groupping_mean_lst + groupping_max_lst

In [10]:
# case_id를 기준으로 그룹화하고 각 그룹에서 첫 번째 값을 선택하여 집계
def merge_with_first(df1: pl.DataFrame, df2: pl.DataFrame, on: str):
    merged_df = df1.join(df2.group_by(on).first(), on=on, how='left')
    return merged_df

def make_days_pipe(df: pl.DataFrame):
    for col in df.columns:
        if col.endswith("D"):
            df = df.with_columns(pl.col(col) - pl.col("date_decision"))
            df = df.with_columns(pl.col(col).dt.total_days())
            df = df.with_columns([pl.col("date_decision").dt.month().alias("month_nb").cast(pl.Int8),
                                      pl.col("date_decision").dt.weekday().alias("weekday_nb").cast(pl.Int8),
                                     ]
                                    )
    return df.drop("date_decision", "MONTH");


## dpdmaxdateyear_596T

- Year when maximum Days Past Due (DPD) occurred for the active contract.
- active 신용 계약에서 최대 연체가 발생한 연도

- 2009년이라는 뜬금없는 row 1개
> 기존의 mode 중 무작위로 선택하는 방식 채택하는방법 사용하는 경우, 2009년이 선택되었을 시, 너무 unique한 row로 나올 것으로 예상

    > 해당 컬럼은 active contract에 대한 컬럼으로 아래의 closed컬럼과의 year 분포가 차이가 있음
    
    > active contract라는 점과 2009년만 따로 one-hot encoding을 하기에는 리스크 있다(2010, 2011년 등 비슷한 연도가 없기 때문)는 점
    
    > 따라서 해당 2009년을 같은 case_id 그룹에서 나온 year로 선택하도록 추가 설정

In [11]:
col = train_static['dpdmaxdateyear_596T']
train_static[['case_id', 'dpdmaxdateyear_596T']].filter(col == 2009.0)

case_id,dpdmaxdateyear_596T
i32,f32
787670,2009.0


In [12]:
train_static = train_static.with_columns(pl.when(pl.col("dpdmaxdateyear_596T") == 2009.0).then(2017.0).otherwise(pl.col("dpdmaxdateyear_596T")).alias('dpdmaxdateyear_596T_changed'))
train_static = train_static.drop('dpdmaxdateyear_596T')
train_static = train_static.with_columns(pl.col('dpdmaxdateyear_596T_changed').alias('dpdmaxdateyear_596T'))

## overdueamountmaxdateyear_2T
- Year when the maximum past due amount occurred for active contracts.
- active 계약에서 최대 지연된 금액이 발생한 연도


- 해당 컬럼에서도 2009년 존재
> 위의 이유처럼 해당 컬럼만 같은 case_id안의 값으로 교체

In [13]:
col = train_static['overdueamountmaxdateyear_2T']
train_static[['case_id', 'overdueamountmaxdateyear_2T']].filter(col == 2009.0)

case_id,overdueamountmaxdateyear_2T
i32,f32
787670,2009.0


In [14]:
train_static = train_static.with_columns(pl.when(pl.col("overdueamountmaxdateyear_2T") == 2009.0).then(2017.0).otherwise(pl.col("overdueamountmaxdateyear_2T")).alias('overdueamountmaxdateyear_2T_changed'))
train_static = train_static.drop('overdueamountmaxdateyear_2T')
train_static = train_static.with_columns(pl.col('overdueamountmaxdateyear_2T_changed').alias('overdueamountmaxdateyear_2T'))

## overdueamountmax_155A / overdueamountmax_35A(closed)
- Maximal past due amount for active contract.
- active 계약에서 발생한 최대 연체 금액


## overdueamountmax2_14A / overdueamountmax2_398A(closed)
- Maximal past due amount for an active contract.
- active 계약에서 발생한 `또 다른` 최대 연체 금액


- 가정 1) max와 max2 컬럼 비교해서 같거나 포함하는 관계? yes!
> 더 큰 범위의 컬럼 `overdueamountmax2_14A` 만 남겨놓고, 나머지 컬럼은 drop

- max amount에 관한 컬럼
> case_id 별로 모은 후, 가장 max 값 선택(+만약 null값이라면 median으로 결측치 채우는 로직 )

In [15]:
train_static = train_static.drop('overdueamountmax_155A', 'overdueamountmax_35A')

## periodicityofpmts_837L
- Frequency of instalments for an active contract.
- active 계약의 할부 지불 빈도

- 단위가 1일, 30일(1달), 90일(2달), 180일(3달), 360(1년) 기준

> case_id별로 지불 빈도가 변경되는 경우(mean 계산시, 소수점)도 있어 기존의 mean 방식으로 적용 후, null값은 아무런 관계없는 값으로 처리(0.0, is_null 생성 X)

In [16]:
col_name_lst = ['periodicityofpmts_837L', 'periodicityofpmts_1102L']

for col_name in col_name_lst:
    subset = train_static.select(['case_id', col_name])

    count = subset.group_by('case_id').agg(pl.mean(col_name).alias(col_name))
    merged_df = subset.join(count, on='case_id', how='left')
    print(merged_df)

    new_col_name = f'{col_name}_right'

    # null 값을 0.0으로 변경
    train_static= train_static.with_columns(pl.when(merged_df[new_col_name].is_null()).then(0.0).otherwise(merged_df[new_col_name]).alias(col_name))

shape: (15_940_537, 3)
┌─────────┬────────────────────────┬──────────────────────────────┐
│ case_id ┆ periodicityofpmts_837L ┆ periodicityofpmts_837L_right │
│ ---     ┆ ---                    ┆ ---                          │
│ i32     ┆ f32                    ┆ f32                          │
╞═════════╪════════════════════════╪══════════════════════════════╡
│ 388     ┆ 30.0                   ┆ 30.0                         │
│ 388     ┆ null                   ┆ 30.0                         │
│ 388     ┆ null                   ┆ 30.0                         │
│ 388     ┆ null                   ┆ 30.0                         │
│ 388     ┆ null                   ┆ 30.0                         │
│ …       ┆ …                      ┆ …                            │
│ 2703454 ┆ null                   ┆ 30.0                         │
│ 2703454 ┆ null                   ┆ 30.0                         │
│ 2703454 ┆ null                   ┆ 30.0                         │
│ 2703454 ┆ null         

In [17]:
train_static[['periodicityofpmts_837L', 'periodicityofpmts_1102L']]

periodicityofpmts_837L,periodicityofpmts_1102L
f32,f32
30.0,0.0
30.0,0.0
30.0,0.0
30.0,0.0
30.0,0.0
30.0,0.0
30.0,0.0
30.0,0.0
30.0,0.0


## totalamount_996A(active), totalamount_6A(closed)
- Total amount of contracts in the credit bureau.
- 신용기관과 관련하여 active/closed 계약의 총 금액
- 0.0 값이 없다는 게 특징


## totaldebtoverduevalue_178A(active), totaldebtoverduevalue_718A(closed)
- Total amount of past due debt on active/closed contracts.
- active/closed 계약에 대한 연체된 총 부채 금액


## totaloutstanddebtvalue_39A(active), totaloutstanddebtvalue_668A(closed)
- Total outstanding debt for active.closed contracts in the credit bureau.
- 신용기관에 기록된 active/closed 계약에 대한 미지급된 총 부채 금액


> active와 closed 를 구분하려고 했으나, closed total amount가 0인데, 부채금액이 >0 인 경우 존재

> active와 closed를 합치고 전부 sum한 값을 aggregation 대표값으로 설정

In [18]:
train_static = train_static.with_columns(
    (pl.col('totalamount_996A').fill_null(0.0) + pl.col('totalamount_6A').fill_null(0.0))
    .cast(pl.Float64).alias('total_amount')
)


train_static = train_static.with_columns(
    (pl.col('totaldebtoverduevalue_178A').fill_null(0.0) + pl.col('totaldebtoverduevalue_718A').fill_null(0.0))
    .cast(pl.Float64)
    .alias('total_overdue_debt_value')
)


train_static = train_static.with_columns(
    (pl.col('totaloutstanddebtvalue_39A').fill_null(0.0) + pl.col('totaloutstanddebtvalue_668A').fill_null(0.0))
    .cast(pl.Float64)
    .alias('total_outstand_debt_value')
)

In [19]:
train_static[['case_id', 'total_amount', 'total_overdue_debt_value', 'total_outstand_debt_value']]

case_id,total_amount,total_overdue_debt_value,total_outstand_debt_value
i32,f64,f64,f64
388,268897.62,0.0,374419.5
388,0.0,0.0,0.0
388,0.0,0.0,0.0
388,0.0,0.0,0.0
388,0.0,0.0,0.0
388,0.0,0.0,0.0
388,0.0,0.0,0.0
388,0.0,0.0,0.0
388,0.0,0.0,0.0


In [20]:
col_name_lst = ['total_amount', 'total_overdue_debt_value', 'total_outstand_debt_value']

for col_name in col_name_lst:
    subset = train_static.select(['case_id', col_name])

    # sum으로 수정
    count = subset.group_by('case_id').agg(pl.sum(col_name).alias(col_name))
    merged_df = subset.join(count, on='case_id', how='left')
    print(merged_df)


    new_col_name = f'{col_name}_right'


    train_static = train_static.with_columns(pl.when(merged_df[new_col_name].is_null()).then(train_static[col_name].median()).otherwise(merged_df[new_col_name]).alias(col_name))

shape: (15_940_537, 3)
┌─────────┬──────────────┬────────────────────┐
│ case_id ┆ total_amount ┆ total_amount_right │
│ ---     ┆ ---          ┆ ---                │
│ i32     ┆ f64          ┆ f64                │
╞═════════╪══════════════╪════════════════════╡
│ 388     ┆ 268897.62    ┆ 268897.62          │
│ 388     ┆ 0.0          ┆ 268897.62          │
│ 388     ┆ 0.0          ┆ 268897.62          │
│ 388     ┆ 0.0          ┆ 268897.62          │
│ 388     ┆ 0.0          ┆ 268897.62          │
│ …       ┆ …            ┆ …                  │
│ 2703454 ┆ 60000.0      ┆ 552978.0           │
│ 2703454 ┆ 109000.0     ┆ 552978.0           │
│ 2703454 ┆ 60000.0      ┆ 552978.0           │
│ 2703454 ┆ 63980.0      ┆ 552978.0           │
│ 2703454 ┆ 0.0          ┆ 552978.0           │
└─────────┴──────────────┴────────────────────┘
shape: (15_940_537, 3)
┌─────────┬──────────────────────────┬────────────────────────────────┐
│ case_id ┆ total_overdue_debt_value ┆ total_overdue_debt_value_ri

In [21]:
train_static[['case_id', 'total_amount', 'total_overdue_debt_value', 'total_outstand_debt_value']]

case_id,total_amount,total_overdue_debt_value,total_outstand_debt_value
i32,f64,f64,f64
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5
388,268897.62,0.0,374419.5


## -= lastupdate_1112D(active)/ lastupdate_388D
- Date of last update for an active contract from credit bureau.
- 신용기관에 기록된 active 계약의 최근 업데이트 날짜

   
> 가장 최근 기록과 관련된 것으로, 해당 컬럼 역시 case_id 그룹안에서 최대 날짜를 선택

> 해당 컬럼은 굳이,,?? 신용기관에 기록에 관한 것이라,,(다른 컬럼과 같이 쓸 순 있지만, 해당 컬럼만은 단독으로 쓰기 어려울 것 같음)


## -= refreshdate_3813885D
- Date when the credit bureau's public sources have been last updated.
- 신용 기관의 공개 소스가 마지막으로 업데이트된 날짜

> 해당 컬럼의 내용은 신용기관의 공개 소스에 대한 날짜로 의미상 필요없을 가능성이 높음

In [22]:
train_static = train_static.drop('lastupdate_1112D', 'lastupdate_388D', 'refreshdate_3813885D')

In [23]:
cat_lst = ['classificationofcontr_13M', 'classificationofcontr_400M', 'contractst_545M', 'contractst_964M', 'description_351M',
          'financialinstitution_382M', 'financialinstitution_591M', 'purposeofcred_426M', 'purposeofcred_874M', 'subjectrole_182M', 'subjectrole_93M', ]
date_lst = ['dateofcredend_289D', 'dateofcredstart_739D', 'dateofcredend_353D', 'dateofcredstart_181D', 'dateofrealrepmt_138D', 'numberofoverdueinstlmaxdat_641D', 'numberofoverdueinstlmaxdat_148D', 
            'overdueamountmax2date_1142D', 'overdueamountmax2date_1002D', 
           ]
groupping_mean_lst = ['annualeffectiverate_199L', 'annualeffectiverate_63L',
                      'contractsum_5085717L','interestrate_508L', 'nominalrate_281L', 'nominalrate_498L', 'numberofcontrsvalue_258L', 'numberofcontrsvalue_358L', 'numberofinstls_320L', 'numberofinstls_229L','numberofoutstandinstls_59L', 
                      'numberofoutstandinstls_520L', 'numberofoverdueinstlmax_1039L', 'numberofoverdueinstls_834L', 'prolongationcount_1120L', 'prolongationcount_599L',
                      'credlmt_230A', 'credlmt_935A', 'debtoutstand_525A', 'debtoverdue_47A','instlamount_768A', 'instlamount_852A', 'monthlyinstlamount_332A', 'monthlyinstlamount_674A', 
                      'outstandingamount_362A', 'outstandingamount_354A', 'overdueamount_659A', 'overdueamount_31A', 'residualamount_488A','residualamount_856A',  
                     'dpdmax_139P', 'dpdmax_757P']
groupping_max_lst = ['numberofoverdueinstlmax_1151L', 'numberofoverdueinstls_725L', 'overdueamountmax2_14A', 'overdueamountmax2_398A']

groupping_total_lst = groupping_mean_lst +groupping_max_lst
print(groupping_total_lst)

t_lst = ['dpdmaxdatemonth_442T','dpdmaxdatemonth_89T', 'dpdmaxdateyear_596T', 'dpdmaxdateyear_896T', 'overdueamountmaxdatemonth_284T', 'overdueamountmaxdatemonth_365T',
         'overdueamountmaxdateyear_2T', 'overdueamountmaxdateyear_994T'
        ]


['annualeffectiverate_199L', 'annualeffectiverate_63L', 'contractsum_5085717L', 'interestrate_508L', 'nominalrate_281L', 'nominalrate_498L', 'numberofcontrsvalue_258L', 'numberofcontrsvalue_358L', 'numberofinstls_320L', 'numberofinstls_229L', 'numberofoutstandinstls_59L', 'numberofoutstandinstls_520L', 'numberofoverdueinstlmax_1039L', 'numberofoverdueinstls_834L', 'prolongationcount_1120L', 'prolongationcount_599L', 'credlmt_230A', 'credlmt_935A', 'debtoutstand_525A', 'debtoverdue_47A', 'instlamount_768A', 'instlamount_852A', 'monthlyinstlamount_332A', 'monthlyinstlamount_674A', 'outstandingamount_362A', 'outstandingamount_354A', 'overdueamount_659A', 'overdueamount_31A', 'residualamount_488A', 'residualamount_856A', 'dpdmax_139P', 'dpdmax_757P', 'numberofoverdueinstlmax_1151L', 'numberofoverdueinstls_725L', 'overdueamountmax2_14A', 'overdueamountmax2_398A']


In [24]:
for col_name in groupping_total_lst:
    if col_name in groupping_max_lst: 
        train_static, df_col = groupby_agg_method(train_static, col_name, 'max')
    else:
        train_static, df_col = groupby_agg_method(train_static, col_name, 'mean')
    
    print(train_static[col_name])

shape: (15_940_537, 3)
┌─────────┬──────────────────────────┬────────────────────────────────┐
│ case_id ┆ annualeffectiverate_199L ┆ annualeffectiverate_199L_right │
│ ---     ┆ ---                      ┆ ---                            │
│ i32     ┆ f32                      ┆ f32                            │
╞═════════╪══════════════════════════╪════════════════════════════════╡
│ 388     ┆ null                     ┆ null                           │
│ 388     ┆ null                     ┆ null                           │
│ 388     ┆ null                     ┆ null                           │
│ 388     ┆ null                     ┆ null                           │
│ 388     ┆ null                     ┆ null                           │
│ …       ┆ …                        ┆ …                              │
│ 2703454 ┆ null                     ┆ null                           │
│ 2703454 ┆ null                     ┆ null                           │
│ 2703454 ┆ null                     ┆ nu

In [25]:
for col_name in t_lst:
    mode_get_value = groupby_mode_value(train_static, col_name)
    print(mode_get_value)


    # One-hot encoding
    train_static = train_static.join(mode_get_value, on='case_id', how='left')
    train_static = train_static.with_columns(pl.when(pl.col('mode_value').is_null()).then(pl.lit(0.0)).otherwise(pl.col('mode_value')).cast(pl.Float64).cast(pl.Int16).alias(col_name))
    train_static = train_static.with_columns(train_static[col_name].to_dummies())
    train_static = train_static.drop('literal') #l iteral 컬럼도 같이 삭제
    train_static = train_static.drop('mode_value')
    train_static = train_static.drop(col_name)

shape: (1_012_589, 2)
┌─────────┬────────────┐
│ case_id ┆ mode_value │
│ ---     ┆ ---        │
│ i32     ┆ f32        │
╞═════════╪════════════╡
│ 2623856 ┆ 12.0       │
│ 981799  ┆ 8.0        │
│ 188763  ┆ 8.0        │
│ 191014  ┆ 4.0        │
│ 1606526 ┆ 5.0        │
│ …       ┆ …          │
│ 1840740 ┆ 8.0        │
│ 1451692 ┆ 7.0        │
│ 2609523 ┆ 7.0        │
│ 207743  ┆ 6.0        │
│ 2672305 ┆ 7.0        │
└─────────┴────────────┘
shape: (1_263_493, 2)
┌─────────┬────────────┐
│ case_id ┆ mode_value │
│ ---     ┆ ---        │
│ i32     ┆ f32        │
╞═════════╪════════════╡
│ 1448203 ┆ 8.0        │
│ 16929   ┆ 1.0        │
│ 1386317 ┆ 2.0        │
│ 1482796 ┆ 8.0        │
│ 1814421 ┆ 11.0       │
│ …       ┆ …          │
│ 1572168 ┆ 10.0       │
│ 1334387 ┆ 9.0        │
│ 1482096 ┆ 5.0        │
│ 1426519 ┆ 3.0        │
│ 1623946 ┆ 7.0        │
└─────────┴────────────┘
shape: (1_263_493, 2)
┌─────────┬────────────┐
│ case_id ┆ mode_value │
│ ---     ┆ ---        │
│ i32    

In [26]:
# masking 컬럼
for col_name in cat_lst:
    train_static = groupby_first_value(train_static, col_name)
    

shape: (15_940_537, 3)
┌─────────┬───────────────────────────┬─────────────────────────────────┐
│ case_id ┆ classificationofcontr_13M ┆ classificationofcontr_13M_right │
│ ---     ┆ ---                       ┆ ---                             │
│ i32     ┆ str                       ┆ str                             │
╞═════════╪═══════════════════════════╪═════════════════════════════════╡
│ 388     ┆ 4408ff0f                  ┆ 4408ff0f                        │
│ 388     ┆ ea6782cc                  ┆ 4408ff0f                        │
│ 388     ┆ a55475b1                  ┆ 4408ff0f                        │
│ 388     ┆ a55475b1                  ┆ 4408ff0f                        │
│ 388     ┆ a55475b1                  ┆ 4408ff0f                        │
│ …       ┆ …                         ┆ …                               │
│ 2703454 ┆ a55475b1                  ┆ ea6782cc                        │
│ 2703454 ┆ a55475b1                  ┆ ea6782cc                        │
│ 2703454 ┆ a55

In [27]:
train_static[cat_lst]

classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
str,str,str,str,str,str,str,str,str,str,str
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""


In [28]:
train_cb_merge = merge_with_first(train_basetable, train_static, 'case_id')
train_cb_merge.pipe(make_days_pipe)

case_id,WEEK_NUM,target,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,credlmt_230A,credlmt_935A,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,debtoutstand_525A,debtoverdue_47A,description_351M,dpdmax_139P,dpdmax_757P,financialinstitution_382M,financialinstitution_591M,instlamount_768A,instlamount_852A,interestrate_508L,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,…,overdueamountmaxdatemonth_365T_11,overdueamountmaxdatemonth_365T_12,overdueamountmaxdatemonth_365T_2,overdueamountmaxdatemonth_365T_3,overdueamountmaxdatemonth_365T_4,overdueamountmaxdatemonth_365T_5,overdueamountmaxdatemonth_365T_6,overdueamountmaxdatemonth_365T_7,overdueamountmaxdatemonth_365T_8,overdueamountmaxdatemonth_365T_9,overdueamountmaxdateyear_2T_0,overdueamountmaxdateyear_2T_2015,overdueamountmaxdateyear_2T_2016,overdueamountmaxdateyear_2T_2017,overdueamountmaxdateyear_2T_2018,overdueamountmaxdateyear_2T_2019,overdueamountmaxdateyear_2T_2020,overdueamountmaxdateyear_994T_0,overdueamountmaxdateyear_994T_2004,overdueamountmaxdateyear_994T_2005,overdueamountmaxdateyear_994T_2006,overdueamountmaxdateyear_994T_2007,overdueamountmaxdateyear_994T_2008,overdueamountmaxdateyear_994T_2009,overdueamountmaxdateyear_994T_2010,overdueamountmaxdateyear_994T_2011,overdueamountmaxdateyear_994T_2012,overdueamountmaxdateyear_994T_2013,overdueamountmaxdateyear_994T_2014,overdueamountmaxdateyear_994T_2015,overdueamountmaxdateyear_994T_2016,overdueamountmaxdateyear_994T_2017,overdueamountmaxdateyear_994T_2018,overdueamountmaxdateyear_994T_2019,overdueamountmaxdateyear_994T_2020,month_nb,weekday_nb
i32,i32,f32,f32,f32,str,str,str,str,f32,f64,f64,i64,i64,i64,i64,i64,f64,f64,str,f64,f64,str,str,f64,f64,f32,f64,f64,f32,f32,i32,f32,f32,f32,f32,f32,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8
0,0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,4
1,0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,4
2,0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,5
3,0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,4
4,0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,5
5,0,0.0,null,null,null,null,null,null,n

In [35]:
train_cb_merge.write_csv('hyeonji.csv')